In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import math
from glob import glob
from scipy.spatial import distance
from scipy import signal
from scipy.fft import fft, fftfreq
from vidstab import VidStab
import os
from pathlib import Path

In [7]:
direc = input("insert video directory")

stabilizer = VidStab()

In [8]:
def slope(p1, p2):
    return (p2[1]-p1[1])/(p2[0]-p1[0])

def closest_node(node, nodes):
    nodes = np.asarray([i[0] for i in nodes])
    closest_index = distance.cdist([node], nodes).argmin()
    return nodes[closest_index]

def get_area(vid, dir_to_save, debug = True):
    # vidname = vid.split('\\')[-1]
    vid = cv2.VideoCapture(vid)
    print(int(vid.get(cv2.CAP_PROP_FPS)))
    ret, frame = vid.read()
    out = cv2.VideoWriter(dir_to_save, cv2.VideoWriter_fourcc('M','J', 'P', 'G'), 5, (np.shape(frame)[1], np.shape(frame)[0]))
    previous_frame = np.zeros(1)
    diffs = []
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15,15))
    while ret:
        ret, frame = vid.read()
        if not ret:
            break
        f1 = frame.copy()
        _, thresh = cv2.threshold(frame, 10, 255, cv2.THRESH_BINARY_INV)
        thresh_g = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
        thresh_g = cv2.medianBlur(thresh_g, 21)
        thresh_torso = thresh_g.copy()
        cv2.erode(thresh_torso, kernel, thresh_torso, iterations = 10)
        cnt_tors, hierarchy_tors = cv2.findContours(thresh_torso, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        cnt_tors = sorted(cnt_tors, key=cv2.contourArea)[-1]
        contours, hierarchy = cv2.findContours(thresh_g, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        # torso = [c for c in contours if cv2.contourArea(c) > 20000][0]
        contours = [c for c in contours if 3000 < cv2.contourArea(c) < 20000]
        contours = sorted(contours, key = cv2.contourArea)[::-1]
        contours.append(cnt_tors)
        contoured_im = cv2.drawContours(frame, contours, -1, (0, 255, 0), 5)
        cv2.drawContours(frame, contours[-1], -1, (255,0,0), 5)
        cv2.drawContours(frame, contours[0], -1, (0, 0, 255), 5)
        centroids = []
        for c in contours:
            M = cv2.moments(c)
            cX = int(M["m10"]/M["m00"])
            cY = int(M["m01"] / M["m00"])
            centroids.append((cX, cY))
        eye_dist = math.dist(centroids[0], centroids[1])
        d_ratio = 200/700.5976020512774
        r_ratio = 250/16277.5
        r = int(cv2.contourArea(contours[0])*r_ratio)
        r = 250
        center_point = ((centroids[0][0]+centroids[1][0])//2, (centroids[0][1]+centroids[1][1])//2)
        closest_torso_pt = closest_node(center_point, contours[-1])
        closest_torso_pt = (int((1-0.8)*center_point[0] + 0.8*closest_torso_pt[0]),int((1-0.8)*center_point[1] +  0.8*closest_torso_pt[1]))
        eye_slope = -1/slope(centroids[0], centroids[1])
        d = int(0.6*math.dist(center_point, centroids[-1])*-1*np.sign(math.dist(center_point, centroids[-1])))
        new_pt = (int(center_point[0]+d), int(center_point[1]+d*eye_slope))
        
        cv2.line(contoured_im, centroids[0], centroids[1], (0,255,0), 3)
        cv2.line(contoured_im, center_point, closest_torso_pt, (0,255,0), 3)
        cv2.circle(contoured_im, closest_torso_pt, r, (0,255,0), 3)
        if debug:
            print("distance to torso: ", math.dist(center_point, centroids[-1]))
            print("sign eyes -> torso: ", np.sign(math.dist(center_point, centroids[-1])))
            print("area of big eye: ", cv2.contourArea(contours[0]))
            print("distance between eyes: ", eye_dist)
            print("distance to heart: ", math.dist(center_point, new_pt))
            print("d_ratio: ", eye_slope)
            print("ratioed distance to heart: ", d)
            plt.imshow(contoured_im)
            plt.show()
        mask1 = np.zeros_like(frame)
        cv2.circle(mask1, closest_torso_pt, r, (255, 255, 255), -1)
        pts_we_care_about = np.where(mask1 == (255, 255, 255))
        cent_of_image = (np.shape(contoured_im)[1]//2, np.shape(contoured_im)[0]//2)
        masked_im = cv2.bitwise_and(f1, mask1)
        M = np.float32([
            [1, 0, cent_of_image[0]-closest_torso_pt[0]],
            [0, 1, cent_of_image[1]-closest_torso_pt[1]]
        ])
        
        translated = cv2.warpAffine(masked_im, M, (np.shape(masked_im)[1], np.shape(masked_im)[0]))
        
        if not previous_frame.any():
            previous_frame = translated
        else:
            diffs.append(np.sum(cv2.subtract(translated, previous_frame)))
            previous_frame = translated
        if not debug:
            # cv2.imshow('area', translated)
            # cv2.waitKey(1)
            out.write(translated)
        if debug:
            print(cent_of_image)
            print(closest_torso_pt)
            plt.imshow(masked_im)
            plt.show()
            return(masked_im)
    cv2.destroyAllWindows()
    vid.release()
    out.release()
    return diffs
    
    
    

In [5]:

# from pathlib import Path

# dirs = os.listdir(direc)
# for d in dirs:
#     dir_to_save = direc+d
#     dir_to_save_to = '\\'.join(dir_to_save.split('\\')[-3:])
#     os.makedirs(dir_to_save_to, exist_ok=True)
#     vids = glob(dir_to_save+'/*.avi')
#     print(dir_to_save_to)

#     for ct, i in enumerate(vids[:]):
#         try:
#             if dbg:
#                 m_img = get_area(i, debug=dbg)
#                 continue
            
#             diffs = get_area(i, dir_to_save_to, debug=dbg)
            
#             vidname = i.split('\\')[-1]
#             stabilizer.stabilize(f'{dir_to_save_to}/{vidname}', f'{dir_to_save_to}/stabilized_{vidname}')
            
#             # detrended = signal.detrend(diffs)
#             # filtered = signal.sosfilt(f, detrended)
#             # N = len(detrended)
#             # T = 1/5
#             # yf = fft(filtered)
#             # xf = fftfreq(N, T)[:N//2]
#             # plt.plot(xf, 2.0/N*np.abs(yf[0:N//2]))
#             # plt.title(f'tadpole {ct}')
#             # plt.show()
#             # print(f'tadpole {ct} bpm', xf[np.where(2.0/N*np.abs(yf[0:N//2]) == max(2.0/N*np.abs(yf[0:N//2])))]*60)
#         except Exception as e:
#             print(f'Error: {str(e)}')
#             print(f"{i} failed")

In [6]:
root_path = input("insert path to experiments")
dbg = False

for path in Path(root_path).rglob('*.avi'):
    p = '\\'.join(str(path).split('\\')[-6:])
    d = '\\'.join(str(path).split('\\')[-6:-1])
    p_stab = p.replace('.avi', '_stabilized.avi')
    os.makedirs(d, exist_ok=True)
    try:
        diffs = get_area(str(path), p, debug=dbg)
        stabilizer.stabilize(p, p_stab)
    except Exception as e:
        print(f'Error: {str(e)}')
        print(f"{p} failed")